In [22]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

In [54]:
places={}
visiter_reviews={}

In [3]:
codes= ['1032580297','1677855640']

In [62]:
for code in codes:
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    url = f'https://pcmap.place.naver.com/restaurant/{code}/review/visitor?entry=pll&from=map&fromPanelNum=1&n_ad_group_type=10&n_query=%EC%97%B0%ED%9D%AC%EB%8F%99%EC%B9%B4%ED%8E%98&timestamp=202311241402&x=126.92933369999999&y=37.572160200912094#'
    driver.get(url)

    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    
    # 장소 이름
    name= soup.find('span', class_='Fc1rA').get_text()
    
    reviews={}
    # menu tag, feature tag
    # 꼭 큰화면으로 실행하기
    tags = driver.find_elements(By.CSS_SELECTOR, 'div.PaWWQ')

    menus= tags[0]
    features= tags[1]

    menus= menus.find_elements(By.CSS_SELECTOR, 'span.Me4yK')
    features= features.find_elements(By.CSS_SELECTOR, 'span.Me4yK')
    
    for i in range(5):
        reviews[f'menu_{i+1}']= menus[i].text
        
    for i in range(5):
        reviews[f'feature_{i+1}']= features[i].text
            
    # short_review_top5        
    top5= driver.find_elements(By.CSS_SELECTOR, 'span.nWiXa')
    for i in range(5):
        reviews[f'short_top{i+1}']= top5[i].text[1:-1]

    # 리뷰 더보기
    for i in range(10):
        more_button = driver.find_element(By.CLASS_NAME, 'fvwqf')
        more_button.click()
        time.sleep(1)

    reviews= driver.find_elements(By.CLASS_NAME, 'zPfVt')
    review_list=[]
    for review in reviews:
        review_list.append(review.text.replace('\n', ' '))

    places[name]= reviews
    visiter_reviews[name]=review_list
    
    driver.quit()

In [63]:
places

{'프로토콜 연희점': <selenium.webdriver.remote.webelement.WebElement (session="38998f479b40a2764e29c1e836aa898c", element="DC70683106B6A5C64B481C7062050D17_element_173")>,
 '보연희': <selenium.webdriver.remote.webelement.WebElement (session="f1a914cc0d2eadbc81f387417ced7f08", element="DAA698C8B00DB7CF3D058D3F63E72C41_element_196")>}

In [64]:
visiter_reviews.keys()

dict_keys(['보연희', '프로토콜 연희점'])

In [17]:
for code in codes:
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    url = f'https://pcmap.place.naver.com/restaurant/{code}/review/visitor?entry=pll&from=map&fromPanelNum=1&n_ad_group_type=10&n_query=%EC%97%B0%ED%9D%AC%EB%8F%99%EC%B9%B4%ED%8E%98&timestamp=202311241402&x=126.92933369999999&y=37.572160200912094#'
    driver.get(url)

    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')


In [5]:
name= soup.find('span', class_='Fc1rA').get_text()
name


'보연희'

In [31]:
visiter_reviews={}
tags = driver.find_elements(By.CSS_SELECTOR, 'div.PaWWQ')
tags

[<selenium.webdriver.remote.webelement.WebElement (session="5e4b469070f7786df485adb82d606aec", element="0B62C674E1760DEEA087C26D6888442B_element_24")>,
 <selenium.webdriver.remote.webelement.WebElement (session="5e4b469070f7786df485adb82d606aec", element="0B62C674E1760DEEA087C26D6888442B_element_25")>]

In [37]:
# 꼭 큰화면으로 실행하기
menus= tags[0]
features= tags[1]
menu_list=[]
feature_list=[]

menus= menus.find_elements(By.CSS_SELECTOR, 'span.Me4yK')
features= features.find_elements(By.CSS_SELECTOR, 'span.Me4yK')

for i in range(5):
#     menu_list.append(menus[i].text)
    visiter_reviews[f'menu_{i+1}']= menus[i].text
for i in range(5):
    visiter_reviews[f'feature_{i+1}']= features[i].text


In [42]:
pd.DataFrame(visiter_reviews, index=[0])
# visiter_reviews

,menu_0,feature_0,menu_1,feature_1,menu_2,feature_2,menu_3,feature_3,menu_4,feature_4,menu_5,feature_5,short_top1,short_top2,short_top3,short_top4,short_top5
0,케이크139,맛221,케이크139,맛221,파르페46,만족도122,과일41,서비스26,딸기케이크32,메뉴11,커피19,목적10,디저트가 맛있어요,특별한 메뉴가 있어요,친절해요,음료가 맛있어요,커피가 맛있어요


In [41]:
# 리뷰 top5
# top5_short=[]
top5= driver.find_elements(By.CSS_SELECTOR, 'span.nWiXa')
for i in range(5):
#     top5_short.append(i.text[1:-1])
    visiter_reviews[f'short_top{i+1}']= top5[i].text[1:-1]
    

In [8]:
import re

menu_dict={}
for input_string in menu_list:
    # 정규 표현식을 사용하여 문자와 숫자 분리
    matches = re.match(r'([a-zA-Z가-힣]+)(\d+)', input_string)
    print(matches)
    # 결과 출력 또는 다른 용도로 사용
    if matches:
        coffee_type, quantity = matches.groups()
        menu_dict[coffee_type]=quantity
menu_dict

<re.Match object; span=(0, 6), match='케이크139'>
<re.Match object; span=(0, 5), match='파르페46'>
<re.Match object; span=(0, 4), match='과일41'>
<re.Match object; span=(0, 7), match='딸기케이크32'>
<re.Match object; span=(0, 4), match='커피19'>
<re.Match object; span=(0, 8), match='생크림케이크17'>
<re.Match object; span=(0, 7), match='아이스크림11'>
<re.Match object; span=(0, 6), match='다쿠아즈10'>
<re.Match object; span=(0, 10), match='딸기생크림케이크10'>
None
None
None
None
None
None


{'케이크': '139',
 '파르페': '46',
 '과일': '41',
 '딸기케이크': '32',
 '커피': '19',
 '생크림케이크': '17',
 '아이스크림': '11',
 '다쿠아즈': '10',
 '딸기생크림케이크': '10'}

In [9]:
feature_dict={}
for feature_string in feature_list:
    matches=re.match(r'([a-zA-Z가-힣]+)(\d+)', feature_string)
    print(matches)
    if matches:
        tag, cnt= matches.groups()
        feature_dict[tag]=cnt
        
feature_dict

<re.Match object; span=(0, 4), match='맛221'>
<re.Match object; span=(0, 6), match='만족도122'>
<re.Match object; span=(0, 5), match='서비스26'>
<re.Match object; span=(0, 4), match='메뉴11'>
<re.Match object; span=(0, 4), match='목적10'>
<re.Match object; span=(0, 3), match='가격9'>
<re.Match object; span=(0, 4), match='음식량8'>
<re.Match object; span=(0, 4), match='분위기8'>
<re.Match object; span=(0, 3), match='예약5'>
<re.Match object; span=(0, 5), match='대기시간4'>
<re.Match object; span=(0, 4), match='청결도3'>
<re.Match object; span=(0, 3), match='방역2'>
None
None


{'맛': '221',
 '만족도': '122',
 '서비스': '26',
 '메뉴': '11',
 '목적': '10',
 '가격': '9',
 '음식량': '8',
 '분위기': '8',
 '예약': '5',
 '대기시간': '4',
 '청결도': '3',
 '방역': '2'}

In [125]:
for i in range(10):
    more_button = driver.find_element(By.CLASS_NAME, 'fvwqf')
    more_button.click()
    time.sleep(1)

20
30
40
50
60
70
80
89
97
107


In [44]:
reviews= driver.find_elements(By.CLASS_NAME, 'zPfVt')

In [45]:
review_list=[]
for review in reviews:
    review_list.append(review.text.replace('\n', ' '))
        
visiter_reviews['visiter_reviews']= review_list

In [48]:
visiter_reviews

{'menu_0': '케이크139',
 'feature_0': '맛221',
 'menu_1': '케이크139',
 'feature_1': '맛221',
 'menu_2': '파르페46',
 'feature_2': '만족도122',
 'menu_3': '과일41',
 'feature_3': '서비스26',
 'menu_4': '딸기케이크32',
 'feature_4': '메뉴11',
 'menu_5': '커피19',
 'feature_5': '목적10',
 'short_top1': '디저트가 맛있어요',
 'short_top2': '특별한 메뉴가 있어요',
 'short_top3': '친절해요',
 'short_top4': '음료가 맛있어요',
 'short_top5': '커피가 맛있어요',
 'visiter_reviews': ['무화과와 피스타치오가 이렇게 잘 어울릴줄 몰랐는데 완전 최고!! 피스타치오 크림이 진하고 꾸덕한데 느끼하지도 않고 너무 맛있음!! 크림 자체가 맛있어서 피스타치오 갸또도 완전 맛있을듯!! 다음엔 갸또로~',
  '무화과 시즌메뉴보고 달려가길 너무 잘한곳❣️ 다쿠아즈 위에 가나슈 얹은 메뉴가 너무 신박한데 존맛\U0001faf6🏻 비쥬얼만보고 케이크인 줄 알았는데 다쿠아즈 시트 무슨일인가요 사장님 천재💯 무화과도 진짜 많이 때려박아주셔서 무 화과덕후들은 무조건 가야되요 집이 멀지만 굳이굳이 찾아갈 이유 이백프로 있던곳 포장해갔던 밤케이크도 진짜 맛있었어요 다음에 갈땐 케이크 전메뉴 먹을게요 \U0001faf6🏻❣️',
  '망고케잌 너무 맛있음 망고로 가득채워진 것 맘에들고 (뭐 가격이 이러하니.. ㅋㅋ) 고구마도 맛있는데 시트 비스켓 부분이 좀 밸런스를 깨는것 같다. 암튼 만족한 케이크!',
  '매우늦었지만 남기는 리뷰...무화과파르페랑 피스타치오무화과다쿠아즈케이크..! 무화과 처돌이에겐 너무 행복해지는 맛이였습니다 헤헤 지금은 딸기시즌인가 보네용, 또 먹으러갈것...!!!!!',
  '케이크가 정말 맛있어요!! 무화과가 신선하고 

In [52]:
rr={}
rr[name]=review_list

In [53]:
pd.DataFrame(rr)

,보연희
0,무화과와 피스타치오가 이렇게 잘 어울릴줄 몰랐는데 완전 최고!! 피스타치오 크림이 ...
1,무화과 시즌메뉴보고 달려가길 너무 잘한곳❣️ 다쿠아즈 위에 가나슈 얹은 메뉴가 너무...
2,망고케잌 너무 맛있음 망고로 가득채워진 것 맘에들고 (뭐 가격이 이러하니.. ㅋㅋ)...
3,매우늦었지만 남기는 리뷰...무화과파르페랑 피스타치오무화과다쿠아즈케이크..! 무화과...
4,케이크가 정말 맛있어요!! 무화과가 신선하고 우유생크림이 고소하고 무화과랑 잘 어울...
5,인기가 많은 곳이라 엄청 시끌벅적했지만 케이크가 존맛! 테이블이 너무 다닥다닥 붙어...
6,무화과 파르페를 먹어보았어요~ 그냥 파르페인줄 알았는데 토치로 구워주는 퍼포먼스가 ...
7,연희동 핫플레이스라서 들러본곳~~계절탓에 딸기케잌은 없었지만~~무화과케잌도 있고 ~...
8,"과일 파르페와 케이크가 맛있는 카페, 연희동 카페 보연희 복숭아 파르페 물렁물렁한 ..."
9,여기 호지차 바밤바 갸또 존맛입니당 굿! 사실 원래 가려던 곳이 있었는데 휴무라서 ...
